In [1]:
save_=True
args = dict(mode='rgb', 
            train=True, 
            comp_info='charades_PDAN', 
            rgb_model_file=None, 
            flow_model_file=None, 
            gpu='0', dataset='charades', 
            rgb_root='../I3D_Feature_Extraction_resnet-main/output/Charades_v1_480', 
            flow_root='no_root', type='original', lr='0.0001', 
            epoch='100', model='PDAN', APtype='map', randomseed='False', load_model='False', 
            batch_size='32', num_channel='512', run_mode='False', feat='False',
            early_stopping=False)

In [2]:
from __future__ import division
import time
import os
import argparse
import sys

import torchvision.models as models
import torch

from tensorboardX import SummaryWriter
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

# sys.path.append('/data/stars/user/rdai/PhD_work/Graph_net')


def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random

# set random seed
if args['randomseed']=="False":
    SEED = 0
elif args['randomseed']=="True":
    SEED = random.randint(1, 100000)
else:
    SEED = int(args['randomseed'])

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print('Random_SEED!!!:', SEED)

from torch.optim import lr_scheduler
from torch.autograd import Variable

import json

import pickle
import math


from utils import *


if str(args['APtype']) == 'wap':
    pass
    #from wapmeter import APMeter
elif str(args['APtype']) == 'map':
    from apmeter import APMeter


batch_size = int(args['batch_size'])


from charades_i3d_per_video import MultiThumos as Dataset
from charades_i3d_per_video import mt_collate_fn as collate_fn

train_split = './data/charades.json'
test_split = './data/charades.json'
# print('load feature from:', args.rgb_root)
# rgb_root = '/Path/to/charades_feat_rgb'
# skeleton_root = '/Path/to/charades_feat_pose'
# flow_root = '/Path/to/charades_feat_flow'
# rgb_of=[rgb_root,flow_root]
classes = 157


def load_data(train_split, val_split, root):
    # Load Data
    print('load data', root)
    if len(train_split) > 0:
        if str(args['feat']) == '2d':
            dataset = Dataset(train_split, 'training', root, batch_size, classes, int(args['pool_step']))
        else:
            dataset = Dataset(train_split, 'training', root, batch_size, classes)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8,
                                                 pin_memory=True, collate_fn=collate_fn)
        dataloader.root = root
    else:

        dataset = None
        dataloader = None

    if str(args['feat']) == '2d':
        val_dataset = Dataset(val_split, 'testing', root, batch_size, classes, int(args['pool_step']))
    else:
        val_dataset = Dataset(val_split, 'testing', root, batch_size, classes)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=2,
                                                 pin_memory=True, collate_fn=collate_fn)
    val_dataloader.root = root

    dataloaders = {'train': dataloader, 'val': val_dataloader}
    datasets = {'train': dataset, 'val': val_dataset}
    return dataloaders, datasets


# train the model
def run(models, criterion, num_epochs=50):
    since = time.time()
    
    if args['early_stopping']:
        print('INFO: Initializing early stopping')
        early_stopping = EarlyStopping()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        probs = []
#         for model, gpu, dataloader, optimizer, sched, model_file in models:
#             train_map, train_loss = train_step(model, gpu, optimizer, dataloader['train'], epoch)
#             prob_val, val_loss, val_map = val_step(model, gpu, dataloader['val'], epoch)
#             probs.append(prob_val)
#             sched.step(val_loss)
#             print('--------Im in for loop')
        model, gpu, dataloader, optimizer, sched, model_file = models[0]
        train_map, train_loss = train_step(model, gpu, optimizer, dataloader['train'], epoch)
        prob_val, val_loss, val_map = val_step(model, gpu, dataloader['val'], epoch)
        probs.append(prob_val)
        sched.step(val_loss)
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/train', train_map, epoch)
        writer.add_scalar('Accuracy/val',  val_map, epoch)
        
        print(val_loss.item(), 'val_loss.item()')
        #TODO: fix
        if args['early_stopping']:
            early_stopping(val_loss.item())
            if early_stopping.early_stop:
                break
    
    writer.close()

def eval_model(model, dataloader, baseline=False):
    results = {}
    for data in dataloader:
        other = data[3]
        
        outputs, loss, probs, _ = run_network(model[0][0], data, 0, baseline)
        fps = outputs.size()[1] / other[1][0]

        results[other[0][0]] = (outputs.data.cpu().numpy()[0].tolist(), probs.data.cpu().numpy()[0].tolist(), data[2].numpy()[0].tolist(), float(fps))
#         results[other[0][0]] = (outputs.data.cpu().numpy()[0], probs.data.cpu().numpy()[0], data[2].numpy()[0], fps)
    return results


def run_network(model, data, gpu, epoch=0, baseline=False):
    inputs, mask, labels, other = data
    # wrap them in Variable
    inputs = Variable(inputs.cuda(gpu))
    mask = Variable(mask.cuda(gpu))
    labels = Variable(labels.cuda(gpu))

    mask_list = torch.sum(mask, 1)
    mask_new = np.zeros((mask.size()[0], classes, mask.size()[1]))
    for i in range(mask.size()[0]):
        mask_new[i, :, :int(mask_list[i])] = np.ones((classes, int(mask_list[i])))
    mask_new = torch.from_numpy(mask_new).float()
    mask_new = Variable(mask_new.cuda(gpu))

    inputs = inputs.squeeze(3).squeeze(3)
    
    activation = model(inputs, mask_new)
    #TODO
#     activation = model[0](inputs, mask_new)

    
    outputs_final = activation

    #print("outputs_final",outputs_final.size())
    outputs_final = outputs_final[-1]
    #print("outputs_final",outputs_final.size())
    outputs_final = outputs_final.permute(0, 2, 1)  
    probs_f = torch.sigmoid(outputs_final) * mask.unsqueeze(2)
    loss_f = F.binary_cross_entropy_with_logits(outputs_final, labels, size_average=False)
    loss_f = torch.sum(loss_f) / torch.sum(mask)  

    loss = loss_f 

    corr = torch.sum(mask)
    tot = torch.sum(mask)

    return outputs_final, loss, probs_f, corr / tot


def train_step(model, gpu, optimizer, dataloader, epoch):
    model.train(True)
    tot_loss = 0.0
    error = 0.0
    num_iter = 0.
    apm = APMeter()
    for data in dataloader:
        optimizer.zero_grad()
        num_iter += 1

        outputs, loss, probs, err = run_network(model, data, gpu, epoch)
        apm.add(probs.data.cpu().numpy()[0], data[2].numpy()[0])
        error += err.data
        tot_loss += loss.data

        loss.backward()
        optimizer.step()
        
    if args['APtype'] == 'wap':
        train_map = 100 * apm.value()
    else:
        train_map = 100 * apm.value().mean()
    print('train-map:', train_map)
    apm.reset()

    epoch_loss = tot_loss / num_iter

    return train_map, epoch_loss


def val_step(model, gpu, dataloader, epoch):
    model.train(False)
    apm = APMeter()
    tot_loss = 0.0
    error = 0.0
    num_iter = 0.
    num_preds = 0

    full_probs = {}

    # Iterate over data.
    for data in dataloader:
        num_iter += 1
        other = data[3]

        outputs, loss, probs, err = run_network(model, data, gpu, epoch)

        apm.add(probs.data.cpu().numpy()[0], data[2].numpy()[0])

        error += err.data
        tot_loss += loss.data

        probs = probs.squeeze()

        full_probs[other[0][0]] = probs.data.cpu().numpy().T

    epoch_loss = tot_loss / num_iter


    val_map = torch.sum(100 * apm.value()) / torch.nonzero(100 * apm.value()).size()[0]
    print('val-map:', val_map)
#     print(100 * apm.value())
    apm.reset()

    return full_probs, epoch_loss, val_map

Random_SEED!!!: 0


In [3]:
#main

args['model'] = 'PDAN'
print('batch_size:', batch_size)
print('cuda_avail', torch.cuda.is_available())

dataloaders, datasets = load_data(train_split, test_split, args['rgb_root'])

if args['train']:
    num_channel = args['num_channel']
    input_channnel = 2048

    num_classes = classes
    mid_channel=int(args['num_channel'])

    if args['model']=="PDAN":
        print("you are processing PDAN_original")
        from PDAN import PDAN
        # rgb_model = Net(mid_channel, input_channnel, classes)
        stage=1
        block=5
        num_channel=512
        input_channnel=2048
        num_classes=classes
        rgb_model = PDAN(stage, block, num_channel, input_channnel, num_classes)
        pytorch_total_params = sum(p.numel() for p in rgb_model.parameters() if p.requires_grad)
        print('pytorch_total_params', pytorch_total_params)
        #exit()
        print ('stage:', stage, 'block:', block, 'num_channel:', num_channel, 'input_channnel:', input_channnel,
               'num_classes:', num_classes)


    rgb_model=torch.nn.DataParallel(rgb_model)

    if args['load_model']!= "False":
        state_dict = torch.load(str(args['load_model']))

    pytorch_total_params = sum(p.numel() for p in rgb_model.parameters() if p.requires_grad)
    print('pytorch_total_params', pytorch_total_params)
    print('num_channel:', num_channel, 'input_channnel:', input_channnel,'num_classes:', num_classes)
    rgb_model.cuda()

    criterion = nn.NLLLoss(reduce=False)
    lr = float(args['lr'])
    print('lr', lr)
    optimizer = optim.Adam(rgb_model.parameters(), lr=lr)
    lr_sched = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=8, verbose=True)
    print(type(rgb_model), 'type model')


batch_size: 32
cuda_avail True
load data ../I3D_Feature_Extraction_resnet-main/output/Charades_v1_480
split!!!! training


100%|████████████████████████████████████████████████████████████████████████████| 9848/9848 [00:06<00:00, 1485.05it/s]


split!!!! testing


100%|████████████████████████████████████████████████████████████████████████████| 9848/9848 [00:01<00:00, 6422.32it/s]


you are processing PDAN_original


C:\Users\David\Desktop\har\PDAN-main\PDAN.py:89: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
C:\Users\David\Desktop\har\PDAN-main\PDAN.py:90: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.value_conv.weight, mode='fan_out')
C:\Users\David\Desktop\har\PDAN-main\PDAN.py:91: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.query_conv.weight, mode='fan_out')
C:\Users\David\Desktop\har\PDAN-main\PDAN.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(self.rel_t, 0, 1)


pytorch_total_params 6382749
stage: 1 block: 5 num_channel: 512 input_channnel: 2048 num_classes: 157
pytorch_total_params 6382749
num_channel: 512 input_channnel: 2048 num_classes: 157


c:\users\david\appdata\local\programs\python\python37\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


lr 0.0001
<class 'torch.nn.parallel.data_parallel.DataParallel'> type model


In [4]:
%%time
run([(rgb_model, 0, dataloaders, optimizer, lr_sched, args['comp_info'])], criterion, num_epochs=int(args['epoch']))

Epoch 0/99
----------


c:\users\david\appdata\local\programs\python\python37\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\David\Desktop\har\PDAN-main\apmeter.py:108: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  rg = torch.range(1, self.scores.size(0)).float()


train-map: tensor(2.0511)
val-map: tensor(5.2134)
15.3865385055542 val_loss.item()
Epoch 1/99
----------
train-map: tensor(4.5742)
val-map: tensor(6.8618)
14.983321189880371 val_loss.item()
Epoch 2/99
----------
train-map: tensor(5.8262)
val-map: tensor(7.6633)
14.848044395446777 val_loss.item()
Epoch 3/99
----------
train-map: tensor(7.1686)
val-map: tensor(8.2149)
15.007850646972656 val_loss.item()
Epoch 4/99
----------
train-map: tensor(7.3825)
val-map: tensor(8.7203)
14.72641372680664 val_loss.item()
Epoch 5/99
----------
train-map: tensor(7.5304)
val-map: tensor(9.1867)
14.554095268249512 val_loss.item()
Epoch 6/99
----------
train-map: tensor(9.6721)
val-map: tensor(9.4232)
14.4320707321167 val_loss.item()
Epoch 7/99
----------
train-map: tensor(10.5500)
val-map: tensor(9.8080)
14.430047035217285 val_loss.item()
Epoch 8/99
----------
train-map: tensor(11.7393)
val-map: tensor(9.9873)
14.394678115844727 val_loss.item()
Epoch 9/99
----------
train-map: tensor(12.6230)
val-map: tens

Epoch    71: reducing learning rate of group 0 to 7.8125e-07.
17.010156631469727 val_loss.item()
Epoch 72/99
----------
train-map: tensor(42.0012)
val-map: tensor(9.8359)
16.99502944946289 val_loss.item()
Epoch 73/99
----------
train-map: tensor(41.5211)
val-map: tensor(9.8234)
16.983564376831055 val_loss.item()
Epoch 74/99
----------
train-map: tensor(41.2382)
val-map: tensor(9.8231)
16.990293502807617 val_loss.item()
Epoch 75/99
----------
train-map: tensor(42.6675)
val-map: tensor(9.8148)
17.008455276489258 val_loss.item()
Epoch 76/99
----------
train-map: tensor(39.2916)
val-map: tensor(9.8267)
17.02201271057129 val_loss.item()
Epoch 77/99
----------
train-map: tensor(39.8197)
val-map: tensor(9.8191)
17.005578994750977 val_loss.item()
Epoch 78/99
----------
train-map: tensor(41.0853)
val-map: tensor(9.8139)
17.016849517822266 val_loss.item()
Epoch 79/99
----------
train-map: tensor(41.2069)
val-map: tensor(9.8179)
17.025535583496094 val_loss.item()
Epoch 80/99
----------
train-map:

In [5]:
%%time
results = eval_model([(rgb_model, 0, dataloaders, optimizer, lr_sched, args['comp_info'])], dataloaders['val'])

Wall time: 19.4 s


In [6]:
if save_:
    with open('./output/results.txt', 'w') as file:
        file.write(json.dumps(results)) # use `json.loads` to do the reverse

In [7]:
if save_:
    torch.save(rgb_model, './output/rgb_model.pt')